<a href="https://colab.research.google.com/github/Jurgo001/TH_TriTueNhanTao/blob/main/Buoi01_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
from heapq import heappush, heappop
import math

# --- I. THIẾT LẬP CƠ BẢN (THÔNG SỐ TĨNH) ---

O_TRONG = 0 # Giá trị đại diện cho ô trống

# 4 hướng dịch chuyển: (Hàng, Cột)
DOI_HANG = [1, 0, -1, 0]
DOI_COT = [0, -1, 0, 1]

# --- II. CÁC CẤU TRÚC DỮ LIỆU (OOP) ---

class HangDoiUuTien:
    """Lớp hàng đợi ưu tiên sử dụng heap, sắp xếp theo chi phí thấp nhất."""
    # Tác dụng Code: Cấu trúc cốt lõi của thuật toán Best First Search/Branch and Bound.
    def __init__(self):
        # Dữ liệu Động 1: self.heap (Min-Heap)
        self.heap = []

    def day(self, khoa):
        """Thêm một khóa mới (node) vào heap."""
        heappush(self.heap, khoa)

    def lay(self):
        """Lấy phần tử có chi phí thấp nhất (min) ra khỏi queue."""
        # Tác dụng Code: Luôn lấy ra Node tốt nhất để mở rộng tiếp theo (tối ưu hóa).
        return heappop(self.heap)

    def trong_khong(self):
        """Kiểm tra xem hàng đợi có rỗng không."""
        return not self.heap

class Nut:
    """Cấu trúc đại diện cho một trạng thái (node) của bàn cờ."""
    # Tác dụng Code: Đóng gói trạng thái (ma_tran) cùng với thông tin tìm kiếm (chi_phi, so_buoc).
    def __init__(self, cha, ma_tran, vi_tri_o_trong, chi_phi, so_buoc):
        # Dữ liệu Động 2: Trạng thái cha (self.cha)
        self.cha = cha # Tham chiếu tự động xây dựng nên đường đi giải trong bộ nhớ.

        # Dữ liệu Động 3: Ma trận bàn cờ hiện tại (self.ma_tran)
        self.ma_tran = ma_tran # Thông số cốt lõi, thay đổi khi ô trống (0) được hoán đổi vị trí.

        # Dữ liệu Động 4: Vị trí ô trống (self.vi_tri_o_trong)
        self.vi_tri_o_trong = vi_tri_o_trong # Cập nhật sau mỗi bước di chuyển.

        # Dữ liệu Động 5: Chi phí Heuristic h(n) (self.chi_phi)
        self.chi_phi = chi_phi # Số ô sai vị trí. Node có chi phí này thấp nhất sẽ được mở rộng.

        # Dữ liệu Động 6: Số bước đã đi g(n) (self.so_buoc)
        self.so_buoc = so_buoc # Giá trị g(n) +1 so với Node cha, đo lường độ sâu tìm kiếm.

    # Hàm so sánh: Sắp xếp dựa trên chi phí (số ô sai vị trí) thấp nhất
    def __lt__(self, nut_khac):
        return self.chi_phi < nut_khac.chi_phi

# --- III. CÁC HÀM XỬ LÝ LOGIC ---

def tinh_chi_phi_o_sai(ma_tran, ma_tran_dich, KICH_THUOC) -> int:
    """Tính số lượng ô (trừ ô trống 0) không ở đúng vị trí so với trạng thái đích."""
    # Note: Hàm này nhận KICH_THUOC (thông số động) để tính toán cho mọi kích thước N.
    count = 0
    for i in range(KICH_THUOC):
        for j in range(KICH_THUOC):
            if ma_tran[i][j] != O_TRONG and ma_tran[i][j] != ma_tran_dich[i][j]:
                count += 1
    return count

def tao_nut_moi(ma_tran, vi_tri_o_trong, vi_tri_o_moi, so_buoc, cha, ma_tran_dich, KICH_THUOC) -> Nut:
    """Tạo một node con (child node) sau khi dịch chuyển ô trống."""
    # Tác dụng Code: Tạo trạng thái mới (ma_tran_moi) và tính chi phí (chi_phi) mới.
    ma_tran_moi = copy.deepcopy(ma_tran)
    x1, y1 = vi_tri_o_trong[0], vi_tri_o_trong[1]
    x2, y2 = vi_tri_o_moi[0], vi_tri_o_moi[1]

    ma_tran_moi[x1][y1], ma_tran_moi[x2][y2] = ma_tran_moi[x2][y2], ma_tran_moi[x1][y1]
    chi_phi = tinh_chi_phi_o_sai(ma_tran_moi, ma_tran_dich, KICH_THUOC)

    nut_moi = Nut(cha, ma_tran_moi, vi_tri_o_moi, chi_phi, so_buoc)
    return nut_moi

def in_ma_tran(ma_tran, KICH_THUOC):
    """In ma trận N x N một cách dễ đọc."""
    # Note: Nhận KICH_THUOC để in đúng kích thước (tránh lỗi Index khi KICH_THUOC là động).
    for i in range(KICH_THUOC):
        print(" ".join(f"{ma_tran[i][j]:2}" for j in range(KICH_THUOC)))

def la_an_toan(x, y, KICH_THUOC):
    """Kiểm tra xem tọa độ (x, y) có nằm trong phạm vi ma trận N x N không."""
    return 0 <= x < KICH_THUOC and 0 <= y < KICH_THUOC

def in_duong_di(nut_goc, KICH_THUOC):
    """In đường đi từ node gốc đến node đích (đệ quy)."""
    # Tác dụng Code: Sử dụng tham chiếu cha (self.cha) để truy vết ngược và in ra chuỗi bước giải.
    if nut_goc is None:
        return

    in_duong_di(nut_goc.cha, KICH_THUOC)
    print(f"Bước: {nut_goc.so_buoc}")
    in_ma_tran(nut_goc.ma_tran, KICH_THUOC)
    print()

def tao_trang_thai_dich_chuan(N):
    """Tạo ma trận trạng thái đích chuẩn N x N (số 0 ở góc dưới bên phải)."""
    # Tác dụng Code: Sinh ra trạng thái đích chuẩn cho kích thước N bất kỳ (linh hoạt theo Dynamic Input).
    ma_tran_dich = []
    gia_tri = 1
    for i in range(N):
        hang = []
        for j in range(N):
            hang.append(gia_tri)
            gia_tri += 1
        ma_tran_dich.append(hang)

    ma_tran_dich[N-1][N-1] = O_TRONG
    return ma_tran_dich

def nhap_ma_tran_dau(N):
    """Cho phép người dùng nhập ma trận đầu N x N và tìm vị trí ô trống."""
    # Note: Hàm này đáp ứng yêu cầu Dynamic Input cho trạng thái đầu.
    print(f"\n--- Nhập trạng thái đầu {N}x{N} ---")
    print(f"Nhập {N*N} số, cách nhau bằng dấu cách, sau đó nhấn Enter.")
    print(f"Các số phải là duy nhất và nằm trong khoảng 0 đến {N*N - 1}.")

    ma_tran = []
    vi_tri_o_trong = None

    while True:
        try:
            # Nhập tất cả các số trên một dòng
            du_lieu_nhap = list(map(int, input("Nhập dữ liệu: ").split()))

            if len(du_lieu_nhap) != N * N:
                print(f"Lỗi: Cần nhập đúng {N*N} số.")
                continue

            # Kiểm tra các giá trị hợp lệ (0 đến N*N - 1)
            so_can_co = set(range(N * N))
            so_da_nhap = set(du_lieu_nhap)
            if so_can_co != so_da_nhap:
                print(f"Lỗi: Các số phải là duy nhất và nằm trong khoảng 0 đến {N*N - 1}.")
                continue

            # Chuyển thành ma trận N x N
            k = 0
            for i in range(N):
                hang = du_lieu_nhap[k : k + N]
                ma_tran.append(hang)
                k += N

            # Tìm vị trí ô trống
            for i in range(N):
                for j in range(N):
                    if ma_tran[i][j] == O_TRONG:
                        vi_tri_o_trong = [i, j]
                        break
                if vi_tri_o_trong:
                    break

            return ma_tran, vi_tri_o_trong

        except ValueError:
            print("Lỗi: Vui lòng chỉ nhập các số nguyên.")
        except Exception as e:
             print(f"Đã xảy ra lỗi: {e}")

# --- IV. THUẬT TOÁN BRANCH AND BOUND (Best First Search) ---

def giai_puzzle(trang_thai_dau, vi_tri_o_trong_dau, trang_thai_dich, KICH_THUOC):
    """Giải câu đố N-Puzzle bằng phương pháp Branch and Bound."""

    # Dữ liệu Động 1: hang_doi (Cấu trúc quản lý Node)
    hang_doi = HangDoiUuTien()

    # 2. Tạo node gốc (Root node) - Gắn KICH_THUOC động
    chi_phi_goc = tinh_chi_phi_o_sai(trang_thai_dau, trang_thai_dich, KICH_THUOC)
    nut_goc = Nut(None, trang_thai_dau, vi_tri_o_trong_dau, chi_phi_goc, 0)

    hang_doi.day(nut_goc)

    # 3. Vòng lặp tìm kiếm
    while not hang_doi.trong_khong():
        # Tác dụng Code: Lấy node có chi phí (heuristic) thấp nhất (Best First)
        hien_tai = hang_doi.lay()

        # Kiểm tra điều kiện đích: Nếu chi phí = 0 (tất cả ô đúng vị trí)
        if hien_tai.chi_phi == 0:
            print("--- ĐÃ TÌM THẤY GIẢI PHÁP ---")
            in_duong_di(hien_tai, KICH_THUOC)
            return

        x_o_trong, y_o_trong = hien_tai.vi_tri_o_trong[0], hien_tai.vi_tri_o_trong[1]

        # 4. Phát triển các node con
        for i in range(4): # 4 hướng dịch chuyển
            x_moi = x_o_trong + DOI_HANG[i]
            y_moi = y_o_trong + DOI_COT[i]
            vi_tri_o_moi = [x_moi, y_moi]

            if la_an_toan(x_moi, y_moi, KICH_THUOC):
                # Tác dụng Code: Tạo Node con và tính toán lại chi phí (Dữ liệu Động 2-6 được cập nhật)
                nut_con = tao_nut_moi(
                    hien_tai.ma_tran, hien_tai.vi_tri_o_trong, vi_tri_o_moi,
                    hien_tai.so_buoc + 1, hien_tai, trang_thai_dich, KICH_THUOC
                )
                hang_doi.day(nut_con)

    print("KHÔNG TÌM THẤY GIẢI PHÁP (Có thể do ma trận không khả giải hoặc không gian tìm kiếm quá lớn).")

# --- V. THỰC THI CHÍNH ---

def thuc_thi_chinh():
    # Note: Khối lệnh này quản lý Dynamic Input và Demo.

    # 1. Lấy kích thước bàn cờ (Dynamic Input)
    try:
        N = int(input("Nhập kích thước N của N-Puzzle (ví dụ: 3, 4, 5...): "))
    except ValueError:
        print("Kích thước không hợp lệ, mặc định là 3.")
        N = 3

    if N < 2:
        print("Kích thước phải >= 2.")
        return

    # Dữ liệu Động 7: KICH_THUOC
    KICH_THUOC = N

    # 2. Nhập trạng thái đầu từ người dùng (Dynamic Input)
    trang_thai_dau, vi_tri_o_trong_dau = nhap_ma_tran_dau(KICH_THUOC)

    # 3. Tạo trạng thái đích chuẩn
    trang_thai_dich = tao_trang_thai_dich_chuan(KICH_THUOC)

    # 4. Hiển thị và Giải
    print("\n" + "="*40)
    print("--- BẮT ĐẦU GIẢI N-PUZZLE ---")
    print(f"Kích thước bàn cờ: {KICH_THUOC}x{KICH_THUOC}")
    print("Trạng thái đầu:")
    in_ma_tran(trang_thai_dau, KICH_THUOC)
    print("\nTrạng thái đích:")
    in_ma_tran(trang_thai_dich, KICH_THUOC)
    print("Lưu ý: Với N > 3, thời gian tìm kiếm sẽ rất lâu và có thể bị treo.")
    print("="*40)

    # Gọi phương thức giải - Truyền KICH_THUOC động
    giai_puzzle(trang_thai_dau, vi_tri_o_trong_dau, trang_thai_dich, KICH_THUOC)

thuc_thi_chinh()

Nhập kích thước N của N-Puzzle (ví dụ: 3, 4, 5...): 2

--- Nhập trạng thái đầu 2x2 ---
Nhập 4 số, cách nhau bằng dấu cách, sau đó nhấn Enter.
Các số phải là duy nhất và nằm trong khoảng 0 đến 3.
Nhập dữ liệu: 1 2 3 4
Lỗi: Các số phải là duy nhất và nằm trong khoảng 0 đến 3.
Nhập dữ liệu: 0 1 2 3

--- BẮT ĐẦU GIẢI N-PUZZLE ---
Kích thước bàn cờ: 2x2
Trạng thái đầu:
 0  1
 2  3

Trạng thái đích:
 1  2
 3  0

Đang tìm kiếm giải pháp...
Lưu ý: Với N > 3, thời gian tìm kiếm sẽ rất lâu và có thể bị treo.


In [2]:
import math
from heapq import heappush, heappop # Sử dụng Min-Heap (heapq) để tối ưu O(log n)

# --- I. LỚP ĐỒ THỊ VÀ HÀM CỐT LÕI (OOP) ---

class DoThi:
    """Định nghĩa cấu trúc đồ thị và logic truy cập."""
    def __init__(self, danh_sach_ke):
        # Dữ liệu Động 1: Cấu trúc Đồ thị (Được nhập từ người dùng hoặc lấy từ mẫu)
        self.danh_sach_ke = danh_sach_ke

    def lay_dinh_ke(self, dinh_hien_tai):
        """Trả về [(Đỉnh kề, Trọng số), ...]."""
        return self.danh_sach_ke.get(dinh_hien_tai, [])

    def thuat_toan_a_sao(self, bat_dau, ket_thuc, ham_uoc_luong_h):
        """
        Tìm kiếm A* tối ưu. Công thức: f(n) = g(n) + h(n).
        Note: Đây là trung tâm của thuật toán, nơi dữ liệu động được xử lý.
        """
        # Dữ liệu Động 2: tap_mo_heap (Min-Heap/Hàng đợi Ưu tiên)
        # Tác dụng Code: Luôn giữ đỉnh có f(n) nhỏ nhất ở trên cùng (Tối ưu O(log n) cho việc tìm kiếm).
        tap_mo_heap = [(0 + ham_uoc_luong_h(bat_dau), bat_dau)]

        # Dữ liệu Động 3: tap_dong (Closed Set)
        # Tác dụng Code: Ghi nhớ các đỉnh đã được xử lý xong để tránh lặp lại.
        tap_dong = set()

        # Dữ liệu Động 4: chi_phi_g
        # Tác dụng Code: Lưu g(n) - Chi phí thực tế tối thiểu từ bắt đầu đến đỉnh n. Cập nhật khi tìm thấy đường ngắn hơn.
        chi_phi_g = {dinh: math.inf for dinh in self.danh_sach_ke}
        chi_phi_g[bat_dau] = 0

        # Dữ liệu Động 5: cha
        # Tác dụng Code: Lưu đỉnh cha (parent) để truy vết đường đi tối ưu khi tìm thấy đích.
        cha = {}

        while tap_mo_heap:
            # Tác dụng Code: Lấy đỉnh có f(n) thấp nhất ra khỏi heap.
            f_hien_tai, dinh_hien_tai = heappop(tap_mo_heap)

            if dinh_hien_tai == ket_thuc:
                # Tác dụng Code: Truy vết ngược và in ra đường đi tối ưu.
                duong_di = [ket_thuc]
                current = ket_thuc
                while current in cha:
                    current = cha[current]
                    duong_di.append(current)
                duong_di.reverse()

                print(f'Chi phí thực tế: {chi_phi_g[ket_thuc]}')
                return " -> ".join(duong_di)

            if dinh_hien_tai in tap_dong:
                continue

            # Tác dụng Code: Đưa đỉnh đã xử lý vào tập đóng.
            tap_dong.add(dinh_hien_tai)

            for (dinh_ke, trong_so) in self.lay_dinh_ke(dinh_hien_tai):
                # Tác dụng Code: Tính chi phí g(n) mới khi đi qua dinh_hien_tai.
                chi_phi_g_moi = chi_phi_g[dinh_hien_tai] + trong_so

                # Kiểm tra xem đường đi qua dinh_hien_tai có ngắn hơn đường đi đã biết không
                if chi_phi_g_moi < chi_phi_g.get(dinh_ke, math.inf):

                    # Tác dụng Code: Cập nhật đường đi ngắn hơn (g(n) và cha)
                    chi_phi_g[dinh_ke] = chi_phi_g_moi
                    cha[dinh_ke] = dinh_hien_tai

                    # Dữ liệu Động 6: f_moi
                    # Tác dụng Code: Tính f(n) mới và đẩy vào heap để xếp hàng ưu tiên.
                    f_moi = chi_phi_g_moi + ham_uoc_luong_h(dinh_ke)
                    heappush(tap_mo_heap, (f_moi, dinh_ke))

        return "Không tìm thấy đường đi!"

# --- II. HÀM INPUT ĐỘNG VÀ CHẠY MẪU (DEMO) ---

def lay_du_lieu_mau():
    """Tạo dữ liệu Đồ thị mẫu sẵn có cho chế độ chạy nhanh."""
    print("--- CHẠY CHẾ ĐỘ DEMO MẪU ---")

    ds_ke_mau = {
        'A': [('B', 6), ('F', 3)], 'B': [('C', 3), ('D', 2)],
        'C': [('D', 1), ('E', 5)], 'D': [('B', 2), ('E', 8)],
        'E': [('Z', 5)], 'F': [('G', 1)], 'G': [('I', 3)],
        'H': [('I', 2)], 'I': [('Z', 3)], 'Z': []
    }
    heuristic_mau = {
        'A': 10, 'B': 8, 'C': 5, 'D': 4, 'E': 3,
        'F': 10, 'G': 6, 'H': 5, 'I': 2, 'Z': 0
    }

    # Dữ liệu Động 7: Hàm Heuristic (Đóng gói Heuristic mẫu)
    def ham_uoc_luong_h_mau(n):
        return heuristic_mau.get(n, math.inf)

    return ds_ke_mau, ham_uoc_luong_h_mau, 'A', 'Z'

def nhap_du_lieu_do_thi():
    """Lấy đồ thị và heuristic từ người dùng (Dynamic Input)."""
    # Note: Đây là khối Dynamic Input quan trọng, cho phép thầy thay đổi các thông số.
    print("--- 1. NHẬP CẤU TRÚC ĐỒ THỊ (DYNAMIC INPUT) ---")

    ds_ke = {}
    while True:
        line = input("Cạnh (> END để kết thúc): ").upper()
        if line == 'END': break
        # Tác dụng Code: Phân tích cú pháp input của người dùng và xây dựng ds_ke.
        try:
            goc, dich, trong_so_str = line.split()
            trong_so = int(trong_so_str)
            if goc not in ds_ke: ds_ke[goc] = []
            if dich not in ds_ke: ds_ke[dich] = []
            ds_ke[goc].append((dich, trong_so))
        except:
            print("Lỗi. Nhập lại 'A B 6' (Ví dụ: Đỉnh_Gốc Đỉnh_Đích Trọng_số).")

    tat_ca_dinh = list(ds_ke.keys())

    print("\n--- 2. NHẬP HEURISTIC h(n) (DYNAMIC INPUT) ---")
    heuristic = {}
    for dinh in tat_ca_dinh:
        while True:
            try:
                chi_phi_str = input(f"h({dinh}) > ")
                # Dữ liệu Động 8: Giá trị Heuristic (Được gán trực tiếp bởi người dùng)
                heuristic[dinh] = int(chi_phi_str)
                break
            except ValueError:
                print("Lỗi: Vui lòng nhập số nguyên.")

    # Dữ liệu Động 7 (Hàm Heuristic - Tái định nghĩa cho input người dùng)
    def ham_uoc_luong_h(n):
        return heuristic.get(n, math.inf)

    return ds_ke, ham_uoc_luong_h, None, None

def thuc_thi_chinh():
    """Hàm Main để Demo."""
    # ... [Logic lựa chọn chế độ M/N] ...
    print("="*50)
    print("--- THUẬT TOÁN A* TÌM ĐƯỜNG ĐI NGẮN NHẤT ---")
    print("DEMO: Nhập 'M' (Mẫu) để chạy nhanh hoặc 'N' (Nhập) để nhập Dynamic Input.")
    print("="*50)

    lua_chon = input("Chọn chế độ (M/N): ").upper()

    if lua_chon == 'M':
        ds_ke, ham_h_input, bat_dau, ket_thuc = lay_du_lieu_mau()
    else:
        ds_ke, ham_h_input, _, _ = nhap_du_lieu_do_thi()

        if not ds_ke: return
        bat_dau = input("\nNhập Đỉnh Bắt đầu: ").upper()
        ket_thuc = input("Nhập Đỉnh Kết thúc: ").upper()

    if bat_dau not in ds_ke or ket_thuc not in ds_ke: return

    do_thi = DoThi(ds_ke)

    # Tác dụng Code: Hiển thị trạng thái giải để dễ dàng đối chiếu với thầy
    print("\n" + "="*50)
    print("--- BẮT ĐẦU GIẢI ---")
    if lua_chon == 'M':
         print("Đồ thị đang giải:")
         for k, v in ds_ke.items():
             print(f"  {k}: {v} (h={ham_h_input(k)})")
         print("-" * 20)

    ket_qua = do_thi.thuat_toan_a_sao(bat_dau, ket_thuc, ham_h_input)
    print(f"Đường đi tìm thấy: {ket_qua}")
    print("="*50)

thuc_thi_chinh()

--- THUẬT TOÁN A* TÌM ĐƯỜNG ĐI NGẮN NHẤT ---
DEMO: Nhập 'M' (Mẫu) để chạy nhanh hoặc 'N' (Nhập) để nhập Dynamic Input.
Chọn chế độ (M/N): m
--- CHẠY CHẾ ĐỘ DEMO MẪU ---

--- BẮT ĐẦU GIẢI ---
Đồ thị đang giải:
  A: [('B', 6), ('F', 3)] (h=10)
  B: [('C', 3), ('D', 2)] (h=8)
  C: [('D', 1), ('E', 5)] (h=5)
  D: [('B', 2), ('E', 8)] (h=4)
  E: [('Z', 5)] (h=3)
  F: [('G', 1)] (h=10)
  G: [('I', 3)] (h=6)
  H: [('I', 2)] (h=5)
  I: [('Z', 3)] (h=2)
  Z: [] (h=0)
--------------------
Chi phí thực tế: 10
Đường đi tìm thấy: A -> F -> G -> I -> Z
